# **Capstone Project - The Battle of Neighborhoods**
---

## Introduction
**Background**:

* The café and coffee shop industry is an important industry in terms of employment and contribution to the UK economy, according to "Investigating the Success of Independent Coffee Shops and Cafes in the UK: Findings from a Pilot Study" (by Jacqueline Douglas, Alexander Douglas, Michele Cano, David Moyes). 
* However, sustainability in terms of longevity is an issue. Despite the low barriers to entry into the industry, cafés are a very high risk business and most start-ups fail. According to the *Office for National Statistics*, only 34.6% of accommodation and food services survive longer than five years (*Office for National Statistics, 2016*).
* It's clear that to survive coffee shops owners need to have some feedback from their clients. The feedback should be analyzed (in this research with the help of Machine Learning) and interpreted, as with this information, entrepreneurs can react and change their coffee shops according to their customers opinions and prosper in this business area.

**Business Problem**:

In this research, the Foursquare Rating has been chosen as the measure of the customers' loyalty for the venues, which have placed their menus on the Foursquare Website.
The aim of this research project is to determine if the connection between Foursquare Rating (customers loyalty measure) and availability of the most popular items in a coffee shop menu exists (also the prices for these most popular items are being taken into consideration).



## Data Sources

* Foursquare Rating is based on a number of signals that from social data mines (likes and dislikes, and positive versus negative tips). It's going to be used as the main measure of the project, as the goal is to find the connection between this metric and availability in the published menus the most popular of the coffee shop items.
* Published on the Foursquare, coffee shop Menus with the items and their prices.
* Foursquare Geospatial Data - latitude and longitude of the venues.
* Some additional information: number of the coffee shop photos, number of guest tips, etc.
* Wikipedia page with the list of the districts of London ordered by population density, based on population estimates for 2019.
https://en.wikipedia.org/wiki/List_of_English_districts_by_population_density
